# SOC Prompt Injection Testing Toolkit - Demo

This notebook demonstrates how to use the SOC Prompt Injection Testing Toolkit to evaluate LLM vulnerabilities in Security Operations Center scenarios.

**Author:** Shivang Patel  
**Institution:** University of Adelaide  
**Version:** 1.0  

---

## 📋 What This Notebook Does

1. Shows how to load and explore the dataset
2. Demonstrates running a single test case
3. Shows how to run batch tests
4. Provides examples of analyzing results
5. Visualizes vulnerability patterns

---

## 🔧 Step 1: Setup and Imports

First, let's import the necessary libraries and modules.

In [ ]:
import sys
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Add src directory to path
sys.path.insert(0, '../src')

# Import our toolkit modules
import utils
from test_soc_llm import create_model, run_single_test, evaluate_response

# Display settings
pd.set_option('display.max_colwidth', 100)
sns.set_style('whitegrid')

print("✓ All imports successful!")

## 📂 Step 2: Load Configuration and Scenarios

Load the configuration file and the SOC scenarios dataset.

In [ ]:
# Load configuration
try:
    config = utils.load_config('../config.yaml')
    print("✓ Configuration loaded successfully")
    print(f"  Configured models: {list(config['models'].keys())}")
except FileNotFoundError:
    print("✗ Config file not found. Please copy config.yaml.example to config.yaml")
    print("  and add your API keys.")

# Load scenarios
scenarios = utils.load_scenarios('../data/soc_scenarios.json')
print(f"\n✓ Scenarios loaded successfully")
print(f"  SOC Roles: {len(scenarios['soc_roles'])}")
print(f"  Attack Types: {len(scenarios['attack_types'])}")
print(f"  Test Cases: {len(scenarios['test_cases'])}")

## 🔍 Step 3: Explore the Dataset

Let's examine the SOC roles and attack types in our dataset.

In [ ]:
# Display SOC Roles
print("\n" + "="*70)
print("SOC ROLES")
print("="*70)

for role in scenarios['soc_roles']:
    print(f"\n{role['role_name']} ({role['role_id']})")
    print(f"  Description: {role['description']}")
    print(f"  System Prompt Length: {len(role['system_prompt'])} characters")

In [ ]:
# Display Attack Types
print("\n" + "="*70)
print("ATTACK TYPES")
print("="*70)

for attack in scenarios['attack_types']:
    print(f"\n{attack['attack_name']} ({attack['attack_id']})")
    print(f"  {attack['description']}")

In [ ]:
# Create summary DataFrame of test cases
test_summary = pd.DataFrame([
    {
        'Scenario ID': tc['scenario_id'],
        'SOC Role': tc['soc_role'],
        'Attack Type': tc['attack_type'],
        'Severity': tc['severity']
    }
    for tc in scenarios['test_cases']
])

print("\n" + "="*70)
print("TEST CASES OVERVIEW")
print("="*70)
print(test_summary)

# Show distribution
print("\nTest Cases by Attack Type:")
print(test_summary['Attack Type'].value_counts())

## 🧪 Step 4: Run a Single Test (Example)

Let's run one test case to see how it works in detail.

**Note:** Make sure you have:
1. Configured your API keys in `config.yaml`
2. For Ollama models: Started the Ollama server with `ollama serve`

In [ ]:
# Select a test case (let's use Scenario 1 - Direct Injection)
test_case = utils.get_test_case_by_id(scenarios, 1)

print("Selected Test Case:")
print(f"  Scenario ID: {test_case['scenario_id']}")
print(f"  SOC Role: {test_case['soc_role']}")
print(f"  Attack Type: {test_case['attack_type']}")
print(f"  Severity: {test_case['severity']}")
print(f"\nInput Data (first 500 chars):")
print(test_case['input_data'][:500] + "...")
print(f"\nExpected Behavior:")
print(test_case['expected_behavior'])

In [ ]:
# Initialize a model (change 'gemini' to your preferred model)
model_id = 'gemini'  # or 'ollama_gemma', 'ollama_mistral', etc.

try:
    # Check if model is enabled and configured
    if not utils.validate_api_key(config, model_id):
        print(f"⚠ Warning: {model_id} API key not configured")
        print("Please configure your API key in config.yaml")
    else:
        # Create model interface
        model = create_model(model_id, config)
        print(f"✓ Model initialized: {model_id}")
        
        # Get SOC role
        soc_role = utils.get_soc_role(scenarios, test_case['soc_role'])
        
        # Run the test
        print("\n🧪 Running test...")
        result = run_single_test(model, test_case, soc_role, config)
        
        # Display results
        print("\n" + "="*70)
        print("TEST RESULT")
        print("="*70)
        print(f"Vulnerability Score: {result['vulnerability_score']}/5")
        print(f"Security Breach: {result['security_breach']}")
        print(f"Response Time: {result['response_time']:.2f}s")
        print(f"\nModel Response:")
        print(result['response'][:1000])  # First 1000 chars
        print(f"\nAnalysis:")
        for key, value in result['analysis'].items():
            print(f"  {key}: {value}")
        
except Exception as e:
    print(f"✗ Error running test: {e}")

## 🔬 Step 5: Run Multiple Tests

Now let's run tests on multiple scenarios to see patterns.

**Warning:** This cell will make multiple API calls. Adjust the number of tests as needed.

In [ ]:
# Run a subset of tests (first 5 scenarios)
results = []

test_cases_subset = scenarios['test_cases'][:5]  # Test first 5 scenarios

try:
    if utils.validate_api_key(config, model_id):
        model = create_model(model_id, config)
        
        print(f"Running {len(test_cases_subset)} tests on {model_id}...\n")
        
        for i, test_case in enumerate(test_cases_subset, 1):
            print(f"[{i}/{len(test_cases_subset)}] Testing scenario {test_case['scenario_id']}...", end=' ')
            
            soc_role = utils.get_soc_role(scenarios, test_case['soc_role'])
            result = run_single_test(model, test_case, soc_role, config)
            results.append(result)
            
            # Status indicator
            score = result['vulnerability_score']
            if score <= 2:
                print("✓ SECURE")
            elif score == 3:
                print("⚠ MODERATE")
            else:
                print("✗ VULNERABLE")
        
        print("\n✓ Tests complete!")
    else:
        print(f"⚠ Skipping batch tests: {model_id} not configured")

except Exception as e:
    print(f"\n✗ Error: {e}")

## 📊 Step 6: Analyze Results

Let's analyze the test results and create visualizations.

In [ ]:
if results:
    # Create DataFrame
    df_results = pd.DataFrame(results)
    
    # Summary statistics
    print("="*70)
    print("RESULTS SUMMARY")
    print("="*70)
    print(f"\nTotal Tests: {len(df_results)}")
    print(f"Average Vulnerability Score: {df_results['vulnerability_score'].mean():.2f}")
    print(f"Security Breaches: {df_results['security_breach'].sum()} ({df_results['security_breach'].sum()/len(df_results)*100:.1f}%)")
    print(f"\nScore Distribution:")
    print(df_results['vulnerability_score'].value_counts().sort_index())
    
    # Display results table
    print("\nDetailed Results:")
    display(df_results[['scenario_id', 'soc_role', 'attack_type', 'vulnerability_score', 'security_breach']])
else:
    print("No results to analyze. Run the tests in Step 5 first.")

In [ ]:
# Visualization 1: Vulnerability Score Distribution
if results:
    plt.figure(figsize=(10, 6))
    
    score_counts = df_results['vulnerability_score'].value_counts().sort_index()
    colors = ['green', 'lightgreen', 'yellow', 'orange', 'red']
    
    plt.bar(score_counts.index, score_counts.values, color=[colors[i-1] for i in score_counts.index])
    plt.xlabel('Vulnerability Score', fontsize=12)
    plt.ylabel('Number of Tests', fontsize=12)
    plt.title(f'Vulnerability Score Distribution - {model_id}', fontsize=14, fontweight='bold')
    plt.xticks(range(1, 6))
    plt.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for i, v in zip(score_counts.index, score_counts.values):
        plt.text(i, v + 0.1, str(v), ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
else:
    print("No results to visualize.")

In [ ]:
# Visualization 2: Vulnerability by Attack Type
if results and len(df_results) > 1:
    plt.figure(figsize=(12, 6))
    
    attack_scores = df_results.groupby('attack_type')['vulnerability_score'].mean().sort_values()
    
    plt.barh(attack_scores.index, attack_scores.values, color='coral')
    plt.xlabel('Average Vulnerability Score', fontsize=12)
    plt.ylabel('Attack Type', fontsize=12)
    plt.title(f'Average Vulnerability by Attack Type - {model_id}', fontsize=14, fontweight='bold')
    plt.xlim(0, 5)
    plt.axvline(x=3, color='red', linestyle='--', alpha=0.5, label='Vulnerable Threshold')
    plt.legend()
    plt.grid(axis='x', alpha=0.3)
    
    # Add value labels
    for i, v in enumerate(attack_scores.values):
        plt.text(v + 0.05, i, f'{v:.2f}', va='center', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
else:
    print("Insufficient data for attack type visualization.")

## 💾 Step 7: Save Results

Save your test results for later analysis.

In [ ]:
if results:
    from datetime import datetime
    
    # Create results directory
    results_dir = utils.ensure_results_dir(config)
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Save JSON
    json_path = results_dir / f'demo_results_{timestamp}.json'
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    print(f"✓ Results saved to: {json_path}")
    
    # Save CSV
    csv_path = results_dir / f'demo_results_{timestamp}.csv'
    df_results.to_csv(csv_path, index=False)
    print(f"✓ Results saved to: {csv_path}")
else:
    print("No results to save.")

## 🎯 Step 8: Next Steps

Now that you've seen how the toolkit works:

### Running Comprehensive Tests

To run all tests from the command line:

```bash
# Test all scenarios on all enabled models
cd ..
python src/test_soc_llm.py --all

# Test specific model
python src/test_soc_llm.py --model gemini

# Test specific attack type
python src/test_soc_llm.py --attack log_poisoning

# Test specific SOC role
python src/test_soc_llm.py --scenario log_analyzer
```

### Customizing the Toolkit

1. **Add new scenarios**: Edit `data/soc_scenarios.json`
2. **Add new models**: Update `config.yaml` and `src/test_soc_llm.py`
3. **Improve evaluation**: Modify `evaluate_response()` in `src/test_soc_llm.py`
4. **Add visualizations**: Extend this notebook

### Documentation

- [Setup Guide](../docs/SETUP_GUIDE.md)
- [Usage Guide](../docs/USAGE.md)
- [Dataset Documentation](../data/README.md)

---

## 📝 Notes

- The evaluation function uses heuristics; results should be manually reviewed
- Some models may produce non-deterministic results; run tests multiple times
- Be mindful of API rate limits when running large batches
- Results are automatically saved to `data/results/`

---

**Questions or issues?** Contact: shivang.patel@adelaide.edu.au